In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
# from pl_bolts.datamodules.mnist_datamodule import MNISTDataModule

import torch
from torch import nn
from torch.nn import functional as F

import wandb
wandb.login()

from src.models import CNN, LitModel
from src.utils import sweep_iteration
from src.dataset import MNISTDataModule

---

In [ ]:
# from torch.optim import SGD
# SGD(self.parameters(), lr=self.lr, nesterov=True, momentum=0.9)

In [ ]:
proj = 'SimSiam-Lightning'

# Setup datamodule. Comes with its own train / val / test dataloader.
mnist = MNISTDataModule('../data/', batch_size=8)
mnist.prepare_data()
mnist.setup()

# Architecture
cnn = CNN(C=mnist.dims[0], num_classes=mnist.num_classes)

In [ ]:
model = LitModel(datamodule=mnist, arch=cnn, lr=1e-3)
wandb_logger = WandbLogger(project=proj)

trainer = Trainer(logger=wandb_logger,    # W&B integration
                  gpus=-1,                # use all GPU's
                  #max_epochs=3            # number of epochs
#                   auto_scale_batch_size=True,
                 )
trainer.fit(model, mnist)

In [ ]:
trainer.test(model, datamodule=mnist)

In [ ]:
wandb.finish()

---

In [ ]:
from src.sweeps import sweep_config

sweep_id = wandb.sweep(sweep_config, project=proj)

In [ ]:
# wandb.agent(sweep_id, function=sweep_iteration, project=proj)